In [5]:
# Installing the necessary libraries
!pip install -q --upgrade transformers datasets peft accelerate

In [6]:
# Importing required libraries
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

In [8]:
# Let's upload the csv file needed
from google.colab import files
uploaded_file = files.upload()

# Get the file name
file_name = list(uploaded_file.keys())[0]

# Read the file to a pandas DataFrame
sales_notes = pd.read_csv(file_name)

# Check the first few rows
sales_notes.head(10)

Saving sales_notes_5000.csv to sales_notes_5000 (1).csv


,id,sales_note,label
0,1,Request to modify packaging dimensions before ...,Custom Spec
1,2,inventory outage pls expedite shipping if poss...,Urgent Need
2,3,Exploring samples for a new product line.,New Product Demand
3,4,Interest shown in 2025x prototype by client,New Product Demand
4,5,Firstorder placed for updated product variant.,New Product Demand
5,6,backorders increasing need quick replenishment,Stocking Issue
6,7,client needs same order as last month for prod...,Repeat Order
7,8,Demo units requested to evaluate updated model...,New Product Demand
8,9,inventory outage pls expedite shipping if poss...,Urgent Need
9,10,Stockout reported for Model X1.,Stocking Issue


In [13]:
# Convert it into Hugging Face Dataset format
sales_hf = Dataset.from_pandas(sales_notes)
print(sales_hf)

# Loading the Phi-1.5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

# Phi-1.5's tokenizer does not have a pad_token defined by default.
'''
Decoder models (like GPT, Phi) usually don't need padding when generating text — but for training a classification model,
we do need padding to batch inputs of different lengths.
'''
tokenizer.pad_token = tokenizer.eos_token

# Checking on tokenizer
print("Tokenizer loaded. Trying a sample encoding:")
print(tokenizer("There's order request for a old stock - Neural Ninjas"))

# Let's define a tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["sales_note"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Applying tokenization
sales_tokenized = sales_hf.map(tokenize_function, batched=True)

# Checking the structure after tokenization
print(sales_tokenized)


Dataset({
    features: ['id', 'sales_note', 'label'],
    num_rows: 5000
})
Tokenizer loaded. Trying a sample encoding:
{'input_ids': [1858, 338, 1502, 2581, 329, 257, 1468, 4283, 532, 47986, 10516, 28121], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'sales_note', 'label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})
